In [1]:
val onlyExport = true

In [2]:
%use serialization
%use dataframe
%use coroutines
@file:DependsOn("com.vladsch.flexmark:flexmark-all:0.64.8")
@file:DependsOn("org.apache.commons:commons-compress:1.26.1")

In [3]:
print("Hola!")

Hola!

In [4]:
import java.io.File

// read all files in the `datasets` directory
val files = File("./datasets").listFiles()!!.map { it.name }
files

[009.전문분야_기술과학_한국어 멀티세션 데이터, 010.전문분야_사회과학_한국어 멀티세션 데이터, 016.행정 문서 대상 기계독해 데이터, 017.뉴스 기사 기계독해 데이터, 018.논문자료 요약 데이터, 019.법률, 규정 (판결서, 약관 등) 텍스트 분석 데이터, 021.도서자료 기계독해, 022.요약문 및 레포트 생성 데이터, 023.방송 콘텐츠 대본 요약 데이터, 025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터, 026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터, 027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터, 028.다국어 구어체 번역 병렬 말뭉치 데이터, 029.대규모 구매도서 기반 한국어 말뭉치 데이터, 030.웹데이터 기반 한국어 말뭉치 데이터, 031.온라인 구어체 말뭉치 데이터, 032.방송콘텐츠 한국어-영어 번역 말뭉치, 034.방송콘텐츠 한국어-유럽어 번역 말뭉치, 036.방송콘텐츠 한국어-아시아어 번역 말뭉치, 044.페르소나 대화, 045.지식검색 대화, 046.공감형 대화, 048.일반상식 문장 생성 데이터, 053.한국어-다국어(영어 제외) 번역 말뭉치(기술과학), 054.한국어-다국어 번역 말뭉치(기초과학), 055.한국어-다국어 번역 말뭉치(인문학), 119.국가기록물 대상 초거대AI 학습을 위한 말뭉치 데이터, 121.한국어 성능이 개선된 초거대AI 언어모델 개발 및 데이터, 141.한국어 멀티세션 대화, 150.숫자연산 기계독해 데이터, 152.기술과학 문서 기계독해 데이터, 153.기술과학 요약 데이터, 155.산업정보 연계 주요국 특허 영-한 데이터, 156.전문분야 영-한, 중-한 번역 말뭉치(식품), 157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터, 157.추상 요약 사실성 검증 데이터, NIKLNEWSPAPER_2022_v1.0_JSON.zip, NIKL_DIALOGUE_2020_v1.4.zip, NIKL_DIALOGUE_2021_v1.1.zip, NIKL_DIAL

In [5]:
// export file names as a markdown list then save it
val markdownList = files.joinToString("\n") { "- $it" }
File("datasets.md").writeText(markdownList)

In [6]:
val scope = newFixedThreadPoolContext(
//    Runtime.getRuntime().availableProcessors() / 2,
    8,
    "preprocessor"
).let { CoroutineScope(it) }

In [7]:
import kotlinx.coroutines.*
import kotlinx.coroutines.sync.Mutex
import kotlinx.coroutines.sync.withLock
import java.util.concurrent.ConcurrentHashMap
import kotlin.random.Random
import kotlin.time.Duration

class ProgressBar(val max: Int, var taskName: String, val register: ProgressBarRegistry) {
    val terminalWidth = 100
    var current = 0
    private val currentMutex = Mutex()

    init {
        register.register(this)
    }

    fun step(): Unit = runBlocking {
        currentMutex.withLock {
            current += 1
        }

        register.update()
        if (current >= max) register.unregister(this@ProgressBar)
    }

    fun stepBy(step: Int): Unit = runBlocking {
        currentMutex.withLock {
            current += step.toInt()
        }

        register.update()
        if (current >= max) register.unregister(this@ProgressBar)
    }

    fun close() {
        register.unregister(this)
    }
    
    fun render() = buildString {
        val len = (current.toDouble() / max * terminalWidth).also { 
            if (it.isNaN()) return@buildString
        }.roundToInt()
        
        if (len < 0 || len > terminalWidth) return@buildString
        
        append(taskName)
        append("\t[")
        append("■".repeat(len))
        append("□".repeat(terminalWidth - len))
        append("]\t(${current}/${max} ${String.format("%.2f", ((current) / max.toDouble()) * 100)}%)")
    }.trim() // .let { htmlResult("<span>${it}</span>") }
}

class ProgressBarRegistry() {
    private val progressBars = ConcurrentHashMap<ProgressBar, Int>()
    private val random = Random(System.currentTimeMillis())

    fun register(progressBar: ProgressBar) {
        progressBars.set(progressBar, random.nextInt().absoluteValue)
//        DISPLAY(progressBar.render().withId(progressBars[progressBar]!!.toString()), id=progressBars[progressBar]!!.toString())
//        this.update()
    }

    fun unregister(progressBar: ProgressBar) {
        progressBars.remove(progressBar)
    }

    fun update() {
        progressBars.forEach { progressBar ->
            // just print it
            if (progressBar.key.max < 100) println(progressBar.key.render())
            else {
                val threshold = (progressBar.key.max.toDouble() / 10).let { 
                    if (it.isNaN()) -1.0 else it
                }.roundToInt()

                if (progressBar.key.current % threshold == 0) println(progressBar.key.render())
            }
//            UPDATE_DISPLAY(progressBar.key.render().withId(progressBars[progressBar.key]!!.toString()), id = progressBars[progressBar.key]!!.toString())
        }
    }
}

class ProgressBarIterable<T>(val list: List<T>, val taskName: String, val registry: ProgressBarRegistry): Iterable<T> {
    private val progressBar = ProgressBar(list.count(), taskName, registry)
    private val iter = list.iterator()
    
    override fun iterator(): Iterator<T> = object : Iterator<T> {
        override fun hasNext(): Boolean = iter.hasNext()
        override fun next(): T = iter.next().also { progressBar.step() }
    }

    fun close() {
        progressBar.close()
    }
}

val registry = ProgressBarRegistry()

fun <T> List<T>.withProgressBar(taskName: String): Iterable<T> = ProgressBarIterable<T>(this, taskName, registry)

In [8]:
import com.vladsch.flexmark.html2md.converter.FlexmarkHtmlConverter
import org.apache.commons.compress.archivers.zip.ZipArchiveEntry
import org.apache.commons.compress.archivers.zip.ZipFile
import java.io.File
import java.io.InputStream
import java.util.zip.ZipEntry
import kotlin.io.path.createTempDirectory
import kotlin.io.path.outputStream

// Let's process the first file, step by step

// first, declare some utils

interface Preprocessor {
    val type: PreprocessorType
    
    fun read(file: File): DataFrame<*>
    fun preprocess(data: DataFrame<*>): DataFrame<*>
}

abstract class PreprocessorWithoutPreprocessing: Preprocessor {
    abstract override fun read(file: File): DataFrame<*>
    
    override fun preprocess(data: DataFrame<*>): DataFrame<*> {
        return data
    }
}

enum class PreprocessorType {
    CORPUS, CORPUS_SPOKEN, TASK_SUMMARIZATION, TASK_TRANSLATION,
    CHAT, DIALECT
}

val converter = FlexmarkHtmlConverter.builder().apply { set(FlexmarkHtmlConverter.SETEXT_HEADINGS, false) }.build()
val correctPunctuation = setOf('.', '!', '?', '"', "'")
val commonGrammarMistakes = mapOf(
    "몇일" to "며칠",
    "는 커녕" to "는커녕",
    "은 커녕" to "은커녕",
    " 라고" to "라고",
    " 부터" to "부터",
    " 투성이" to "투성이",
    " 짜리" to "짜리",
    " 어치" to "어치",
    "할려고" to "하려고",
    "어떻해" to "어떡해",
    "어짜피" to "어차피",
    "곰곰히" to "곰곰이",
    "요세" to "요새",
    "금새" to "금세",
    "왠" to "웬",
    "웬지" to "왠지",
    "오랫만에" to "오랜만에",
    "일일히" to "일일이",
    "희안하다" to "희한하다",
    "희안한" to "희한한",
    "희안하게" to "희한하게",
    "않된" to "안 된",
    "돼었" to "되었",
    "돼고" to "되고",
    "어의없" to "어이없",
    "어의가 없" to "어이가 없",
    "불리우다" to "불리다",
    "불리우는" to "불리는",
    "매일마다" to "매일",
    "깨끗히" to "깨끗이",
    "역활" to "역할",
    "순조로와" to "순조로워",
    "알맞는" to "알맞은",
    "여지껏" to "여태껏",
    "유도심문" to "유도신문",
    "도데체" to "도대체",
    "거에요" to "거예요",
    "궂이" to "굳이",
    "구지" to "굳이",
    "것 뿐" to "것뿐",
    "치루다" to "치르다",
    "치뤘다" to "치렀다",
    "치뤄" to "치러",
)

val commonGrammarMistakesStatistics = commonGrammarMistakes.map { mistake -> mistake.key to (Mutex() to 0) }.toMap().toMutableMap()

fun standardNormalizeText(text: String): String = runBlocking {
    converter.convert(text.trim().replace("\t", " ").replace("  ", " ").replace("\n", "<br>")).replace("○", "-").replace("❍", "-").replace("  ", " ").replace(" ", " ").replace("\n\n", "\n").replace("<br />", "\n").trim().let {
        commonGrammarMistakes.entries.fold(it) { acc, mistake ->
            acc.replace(mistake.key, mistake.value).also {
                if (it != acc) commonGrammarMistakesStatistics[mistake.key]!!.first.withLock {
                    commonGrammarMistakesStatistics[mistake.key] = commonGrammarMistakesStatistics[mistake.key]!!.first to commonGrammarMistakesStatistics[mistake.key]!!.second + 1
                }
            }
        }
    }
}

/**
 * @return true if the text is valid, otherwise false
 */
fun standardVaildText(text: String, isSentence: Boolean = false): Boolean = text.trim().let {
    if (it.isEmpty()) return false
    if (it.length < 8) return false
    
    if (isSentence) {
        if (it.last() !in correctPunctuation) return false
        if (it.first() in correctPunctuation) return false
    }
    
    true
}

fun String.addSuffixIfNotExists(suffix: String): String = if (this.endsWith(suffix)) this else this + suffix

val directMapping = mapOf<String, File.() -> List<File>>(
    "기계독해" to { listOf(this.resolve("기계독해분야")) },
    "도서자료 요약" to { listOf(this.resolve("Training")) },
    "문서요약 텍스트" to { listOf(this.resolve("Training")) },
)

fun File.auto1(): File =  if (this.listFiles()!!.map { it.name }.contains("01.데이터")) this.resolve("01.데이터")
                          else if (this.listFiles()!!.map { it.name }.contains("1.데이터")) this.resolve("1.데이터")
                          else if (this.listFiles()!!.map { it.name }.contains("3.개방데이터")) this.resolve("3.개방데이터").auto1()
                          else this.resolve("01-1.정식개방데이터")

fun File.auto2() = this.listFiles()!!.first { it.name.contains("Training") }

fun File.auto3(raw: Boolean) = if (raw) this.listFiles()!!.filter { it.name.contains("원천데이터") } else this.listFiles()!!.filter { it.name.contains("라벨링데이터") }

fun File.auto(raw: Boolean = false) = this.let { 
    if (directMapping.containsKey(this.name)) directMapping[this.name]!!.invoke(this)
    else this.auto1().auto2().auto3(raw)
}.map { it.walk().toList() }.flatten().filter { it.extension == "zip" }

fun List<File>.zipsJsonCount(): Int = this.map { zipPath ->
    ZipFile.builder().setFile(zipPath).get().use { zipFile ->
        zipFile.entries.toList().count() { entry -> entry.name.endsWith(".json") }
    }
}.sum()

//fun <T: Any> List<File>.zipsJsonWalk(block: (zipPath: String, zipFile: String, stream: InputStream) -> T?): List<T> =
//    ProgressBar(this.zipsJsonCount(), "Zip Walk", registry).let {
//        this.flatMapIndexed { zipIdx, zipPath ->
//            ZipFile.builder().setFile(zipPath).get().use { zipFile ->
//                zipFile.entries.asSequence().filter { f -> f.name.endsWith(".json") }.mapIndexedNotNull() { idx, entry ->
//                    zipFile.getInputStream(entry).use { inputStream ->
//                        runCatching {
//                            block(zipPath.toString(), entry.name, inputStream)
//                        }.getOrNull()
//                    }.apply {
//                        it.taskName = "Zip Walk(now ...${
//                            String.format("%.20s", zipPath.toString().reversed()).reversed()
//                        } ${
//                            String.format("%.20s", entry.name.toString().reversed()).reversed()
//                        })"
//                        it.step()
//                    }
//                }.toList()
//            }
//        }
//    }

fun <T: Any> List<File>.zipsJsonWalkNonAsync(block: (zipPath: String, zipFile: String, stream: InputStream) -> T?): List<T> =
    ProgressBar(this.zipsJsonCount(), "Zip Walk", registry).let {
        this.flatMapIndexed { zipIdx, zipPath ->
            ZipFile.builder().setFile(zipPath).get().use { zipFile ->
                zipFile.entries.asSequence().filter { f -> f.name.endsWith(".json") }
                    .mapIndexed { idx, entry ->
                        zipFile.getInputStream(entry).use { inputStream ->
                            runCatching {
                                block(zipPath.toString(), entry.name, inputStream)
                            }.onFailure { 
                                it.printStackTrace()
                            }.getOrNull()
                        }.apply {
                            it.taskName = "Zip Walk(now ...${
                                String.format("%.20s", zipPath.toString().reversed()).reversed()
                            } ${
                                String.format("%.20s", entry.name.toString().reversed()).reversed()
                            })"
                            it.step()
                        }
                    }.toList().filterNotNull()
            }
        }
    }

fun <T: Any> List<File>.zipsJsonWalk(lessMemory: Boolean = false, block: (zipPath: String, zipFile: String, stream: InputStream) -> T?): List<T> =
    if (lessMemory) this.zipsJsonWalkAsyncLessMem(block)
    else this.zipsJsonWalkAsync(block)

fun <T: Any> List<File>.zipsJsonWalkAsync(block: (zipPath: String, zipFile: String, stream: InputStream) -> T?): List<T> = runBlocking {
    ProgressBar(this@zipsJsonWalkAsync.zipsJsonCount(), "Zip Walk", registry).let {
        this@zipsJsonWalkAsync.flatMapIndexed { zipIdx, zipPath ->
            ZipFile.builder().setFile(zipPath).get().use { zipFile ->
                zipFile.entries.asSequence().filter { f -> f.name.endsWith(".json") }
                    .mapIndexed { idx, entry ->
                        scope.async {
                            zipFile.getInputStream(entry).use { inputStream ->
                                runCatching {
                                    block(zipPath.toString(), entry.name, inputStream)
                                }.getOrNull()
                            }.apply {
                                it.taskName = "Zip Walk(now ...${
                                    String.format("%.20s", zipPath.toString().reversed()).reversed()
                                } ${
                                    String.format("%.20s", entry.name.toString().reversed()).reversed()
                                })"
                                it.step()
                            }
                        }
                    }.toList().awaitAll().filterNotNull()
            }
        }
    }
}

fun <T: Any> List<File>.zipsJsonWalkAsyncLessMem(block: (zipPath: String, zipFile: String, stream: InputStream) -> T?): List<T> = runBlocking {
    // first, decompress all files
    val tempDir = createTempDirectory()
    
    ProgressBar(this@zipsJsonWalkAsyncLessMem.zipsJsonCount(), "Zip Decompress", registry).let {
        this@zipsJsonWalkAsyncLessMem.map { zipPath ->
            scope.async {
                ZipFile.builder().setFile(zipPath).get().use { zipFile ->
                    zipFile.entries.asSequence().filter { f -> f.name.endsWith(".json") }.forEach { entry ->
                        zipFile.getInputStream(entry).use { inputStream ->
                            val file = tempDir.resolve(entry.rawName.let {
                                // try utf-8 , if it fails, then use cp949
                                runCatching { String(it, Charsets.UTF_8) }.getOrNull() ?: String(
                                    it,
                                    Charsets.ISO_8859_1
                                )
                            })
                            file.outputStream().use { inputStream.copyTo(it) }.apply { it.step() }
                        }
                    }
                }
            }
        }.awaitAll()
    }
    
    // second, process
    
    ProgressBar(tempDir.toFile().listFiles()!!.count(), "Zip Walk", registry).let {
        tempDir.toFile().listFiles()!!.toList().map{ file ->
            scope.async {
                file.inputStream().use { inputStream ->
                    runCatching {
                        block(file.absolutePath, file.name, inputStream)
                    }.getOrNull()
                }.apply {
                    it.step()
                }
            }
        }.awaitAll().filterNotNull()
    }
    
    listOf()
}

In [9]:
import kotlinx.serialization.Serializable

@Serializable
data class ChatRow(val role: String, val content: String)

In [10]:
import kotlin.time.measureTime
import kotlin.time.measureTimedValue

class FastPreprocessor(val preprocessor: Preprocessor, var batchSize: Int = 10000): Preprocessor by preprocessor {
    override fun preprocess(data: DataFrame<*>): DataFrame<*> = runBlocking {
        if (preprocessor is PreprocessorWithoutPreprocessing) return@runBlocking data
        
//        measureTimedValue {
        val batches = data.chunked(batchSize).toList()
        val progress = ProgressBar(data.rowsCount(), "Batch processing", registry)
        val processed = 
            batches.map { batch -> scope.async { preprocessor.preprocess(batch).apply { 
                progress.stepBy(batchSize)
            } } }.awaitAll()
        
        processed.reduce { acc, df -> acc.concat(df) }
//        }.also { println("Time elapsed: ${it.duration.inWholeMilliseconds}ms") }.value
    }
}

In [11]:
val preprocessors = mutableMapOf<String, MutableMap<PreprocessorType, Preprocessor>>()

fun declarePreprocessor(preprocessor: Preprocessor, forFiles: List<String>, showDatabase: Boolean = false, showExamples: Boolean = false, showInfo: Boolean = true, fast: Boolean = true) {
    val fastProcessor = if (fast) FastPreprocessor(preprocessor) else preprocessor

    forFiles.forEach { preprocessors[it].let { map ->
        if (map == null) preprocessors[it] = mutableMapOf(preprocessor.type to fastProcessor)
        else map[preprocessor.type] = fastProcessor
    } }

    if (onlyExport) return

    val data = forFiles.map { fastProcessor.read(File("./datasets/${it}")) }.reduce { acc, df -> acc.concat(df) }
        .also { println("Count of rows: ${it.rowsCount()}") }

    if (showDatabase) DISPLAY(data.head(100))

    val result = fastProcessor.preprocess(data)

    if (showExamples) {
        DISPLAY(result.shuffle().head(100).let {
            if (showInfo) DataFrame.readJsonStr(
                buildJsonObject {
                    put(when(preprocessor.type) {
                        PreprocessorType.CHAT -> "chat"
                        else -> "text"
                    }, buildJsonObject {
                            put("before_row_count", data.rowsCount())
                            put("after_row_count", result.rowsCount())
                            put("preprocessor", preprocessor::class.simpleName)
                            put("preprocessor_type", preprocessor.type.name)
                        })
                }.toString()
            ).concat(it) else it
        })
    }
}

In [12]:
fun DataFrame<*>.filterColumns(vararg columnsToKeep: String): DataFrame<*> = this.remove(*(this.columnNames() - columnsToKeep).toTypedArray())

In [13]:
fun List<ChatRow>.mergeRepeating(): List<ChatRow> {
    if (this.isEmpty()) return emptyList()

    val merged = mutableListOf<ChatRow>()
    var lastSpeaker = this.first().role
    var lastText = this.first().content

    for ((speaker, text) in this.drop(1)) {
        if (speaker == lastSpeaker) {
            lastText += " $text"
        } else {
            merged.add(ChatRow(lastSpeaker, lastText))
            lastSpeaker = speaker
            lastText = text
        }
    }
    merged.add(ChatRow(lastSpeaker, lastText))

    return merged
}

In [14]:
class PreprocessorForMachineReading : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { "paragraphs"<DataFrame<*>>().first()["context"].let { standardNormalizeText(it as String) } }
            .filter { standardVaildText("text"<String>()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["data"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForMachineReading(),
    listOf("016.행정 문서 대상 기계독해 데이터", "017.뉴스 기사 기계독해 데이터", "021.도서자료 기계독해", "기계독해"),
    showExamples = true
)

In [15]:
class PreprocessorForMachineReadingMath : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { "## ${"title"<String>()}\n ${"passage"<String>()}".let { standardNormalizeText(it) } }
            .filter { standardVaildText("text"<String>()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["data"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForMachineReadingMath(),
    listOf("150.숫자연산 기계독해 데이터"),
    showExamples = true
)

In [16]:
class PreprocessorForMachineReadingTech : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { "context"<String>().let { standardNormalizeText(it) } }
            .filter { standardVaildText("text"<String>()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["dataset"]["context_info"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForMachineReadingTech(),
    listOf("152.기술과학 문서 기계독해 데이터"),
    showExamples = true
)

In [17]:
val translateInstructions = listOf(
    "{a}로 쓰인 내용을 {b}로 번역해 주세요.",
    "{a}에서 {b}로 번역해 주세요.",
    "{a}로 작성된 문장을 {b}로 적어 주세요.",
    "{a} 글을 {b} 언어로 자연스럽게 번역해 주세요.",
    "{a}로 쓰인 글을 {b}로 옮겨 주세요.",
    "{a}로 된 글을 {b}로 바꿔 보세요.",
    "{a}에서 {b}로 자연스럽게 번역해 주세요.",
    "{a} 내용을 {b}로 다시 써 주세요.",
    "{a} 문장을 {b}로 자연스럽게 바꿔 주세요.",
    "{a} 내용을 {b}로 매끄럽게 번역해 주세요.",
    "{a}를 {b}로 해석해 주세요.",
    "이 글을 {b}로 번역해 주세요.",
    "{b}로 옮겨 쓰세요.",
)

In [18]:
class PreprocessorForParrellLanguageTypeA : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_TRANSLATION

    private val languageTable = mapOf(
        "en" to "영어",
        "ko" to "한국어",
        "cn" to "중국어",
        "jp" to "일본어",
    )

    private fun form(index: Int, style: String, sourceLanguage: String, targetLanguage: String, sourceText: String, targetText: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", translateInstructions[index % translateInstructions.size].replace("{a}", sourceLanguage).replace("{b}", targetLanguage) + "\n\n" + sourceText))
        result.add(ChatRow("assistant", targetText))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .add("from") { languageTable["source_language"<String>()]!! }.also { println("from") }
            .add("to") { languageTable["target_language"<String>()]!! }.also { println("to") }
            .filter { standardVaildText("${"source_language"<String>()}"<String>()) && standardVaildText("${"target_language"<String>()}"<String>()) }.also { println("filter") }
//            .explode("from")
//            .explode("to")
            .add("messages") {
                form(
                    index=index(), style = "style"<String>(), sourceLanguage ="from"<String>(), targetLanguage = "to"<String>(), sourceText = ("${"source_language"<String>()}"<String>()).let {
                    standardNormalizeText(it)
                }, targetText = ("${"target_language"<String>()}"<String>()).let {
                    standardNormalizeText(it)
                })
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["data"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

//PreprocessorForParrellLanguage().read(File("./datasets/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터"))

declarePreprocessor(
    PreprocessorForParrellLanguageTypeA(),
    listOf("025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터", "026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터", "156.전문분야 영-한, 중-한 번역 말뭉치(식품)", "157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터"),
    showExamples = true,
    fast = true
)

In [19]:
class PreprocessorForParrellLanguageTypeB : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_TRANSLATION

    private val languageTable = mapOf(
        "en-US" to "영어",
        "ko-KR" to "한국어",
        "fr-FR" to "프랑스어",
        "zh-CN" to "중국어",
        "ja-JP" to "일본어",
        "de-DE" to "독일어",
        "es-ES" to "스페인어",
        "it-IT" to "이탈리아어"
    )

    private fun form(index: Int, sourceLanguage: String, targetLanguage: String, sourceText: String, targetText: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", translateInstructions[index % translateInstructions.size].replace("{a}", sourceLanguage).replace("{b}", "구어체의 ${targetLanguage}") + "\n\n" + sourceText))
        result.add(ChatRow("assistant", targetText))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter {
                this["S_Code"] in languageTable.keys && this["T_Code"] in languageTable.keys && this["원문"] is String && this["최종번역문"] is String
            }
            .filter { standardVaildText("원문"<String>()) && standardVaildText("최종번역문"<String>()) }
            .add("from") { "S_Code"<String>().let { languageTable[it]!! } }
            .add("to") { "T_Code"<String>().let { languageTable[it]!!} }
//            .explode("from")
//            .explode("to")
            .add("messages") {
                form(
                    index=index(),
                    sourceLanguage = "from"<String>(),
                    targetLanguage = "to"<String>(),
                    sourceText = ("원문"<String>()).let { standardNormalizeText(it) },
                    targetText = ("최종번역문"<String>()).let { standardNormalizeText(it) }
                )
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            try {
                val dataFrame = DataFrame.readJson(stream)
                dataFrame
            } catch (e: Exception) {
                null
            }
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

//PreprocessorForParrellLanguageTypeB().read(File("./datasets/028.다국어 구어체 번역 병렬 말뭉치 데이터")).let {
//    // print all languages
//    it["S_Code"].distinct().forEach { println(it) }
//    it["T_Code"].distinct().forEach { println(it) }
//}

declarePreprocessor(
    PreprocessorForParrellLanguageTypeB(),
    listOf("027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터", "028.다국어 구어체 번역 병렬 말뭉치 데이터"),
    showExamples = true
)

In [20]:
class PreprocessorForParrellLanguageTypeC : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_TRANSLATION

    private val languageTable = mapOf(
        "en-US" to "영어",
        "ko-KR" to "한국어",
        "fr-FR" to "프랑스어",
        "zh-CN" to "중국어",
        "ja-JP" to "일본어",
        "de-DE" to "독일어",
        "es-ES" to "스페인어",
        "it-IT" to "이탈리아어",
        "ru-RU" to "러시아어",
    )

    private fun form(index: Int, sourceLanguage: String, targetLanguage: String, sourceText: String, targetText: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", translateInstructions[index % translateInstructions.size].replace("{a}", sourceLanguage).replace("{b}", targetLanguage) + "\n\n" + sourceText))
        result.add(ChatRow("assistant", targetText))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
//            .filter {
//                "S_Code" in this.columnNames() && "T_Code" in this.columnNames() && "원문" in this.columnNames() && "최종번역문" in this.columnNames()
//            }
//            .filter {
//                this["S-Code"] in languageTable.keys && this["T-Code"] in languageTable.keys && this["원문"] is String && this["최종번역문"] is String
//            }
            .filter { standardVaildText("원문"<String>()) && standardVaildText("최종번역문"<String>()) }
            .add("from") { "S-Code"<String>().let { languageTable[it]!! } }
            .add("to") { "T-Code"<String>().let { languageTable[it]!!} }
//            .explode("from")
//            .explode("to")
            .add("messages") {
                form(
                    index=index(),
                    sourceLanguage = "from"<String>(),
                    targetLanguage = "to"<String>(),
                    sourceText = ("원문"<String>()).let { standardNormalizeText(it) },
                    targetText = ("최종번역문"<String>()).let { standardNormalizeText(it) }
                )
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        var bigText = kotlin.io.path.createTempFile()

        println("Merging all json files... temporary file: ${bigText.toFile().absolutePath}")

        val writer = bigText.toFile().bufferedWriter()

        writer.write("[")

        file.auto().zipsJsonWalk { _, _, stream ->
            // json files are too many, so it should be merged as a single json array
            writer.write(stream.bufferedReader().readText().also {
                // if any line which " is included more than 4 times, then it should be removed
                // so throw an exception
                if (it.lines().any { it.count { c -> c == '"' } > 4 }) throw Exception("Too many quotes")
            } + ",")
        }

        writer.write("{}") // to remove the last comma

        writer.write("]")
        writer.close()

        val dataFrame = DataFrame.readJson(bigText.toFile()).let {
            // skip last dummy
            it[0..(it.rowsCount() - 2)]
        }

        return dataFrame
    }
}

//PreprocessorForParrellLanguageTypeB().read(File("./datasets/028.다국어 구어체 번역 병렬 말뭉치 데이터")).let {
//    // print all languages
//    it["S_Code"].distinct().forEach { println(it) }
//    it["T_Code"].distinct().forEach { println(it) }
//}

declarePreprocessor(
    PreprocessorForParrellLanguageTypeC(),
    listOf("032.방송콘텐츠 한국어-영어 번역 말뭉치", "034.방송콘텐츠 한국어-유럽어 번역 말뭉치"),
    showDatabase = false,
    showExamples = true
)

In [21]:
class PreprocessorForParrellLanguageTypeD : PreprocessorWithoutPreprocessing() {
    override val type: PreprocessorType = PreprocessorType.TASK_TRANSLATION

    private fun form(index: Int, sourceLanguage: String, targetLanguage: String, sourceText: String, targetText: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", translateInstructions[index % translateInstructions.size].replace("{a}", sourceLanguage).replace("{b}", targetLanguage) + "\n\n" + sourceText))
        result.add(ChatRow("assistant", targetText))

        return result
    }

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream)
            dataFrame.concat()
                .filter { standardVaildText("07_text"<DataRow<*>>()["1_text"] as String) && standardVaildText("08_translation"<DataRow<*>>()["1_text"] as String) }
                .add("messages") {
                    form(
                        index = index(),
                        sourceLanguage = "02_srcinfo"<DataRow<*>>()["3_language"] as String,
                        targetLanguage = "08_translation"<DataRow<*>>()["2_language"] as String,
                        sourceText = ("07_text"<DataRow<*>>()["1_text"] as String).let { standardNormalizeText(it) },
                        targetText = ("08_translation"<DataRow<*>>()["1_text"] as String).let { standardNormalizeText(it) }
                    )
                }
                .filterColumns("messages")
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

//PreprocessorForParrellLanguageTypeB().read(File("./datasets/028.다국어 구어체 번역 병렬 말뭉치 데이터")).let {
//    // print all languages
//    it["S_Code"].distinct().forEach { println(it) }
//    it["T_Code"].distinct().forEach { println(it) }
//}

declarePreprocessor(
    PreprocessorForParrellLanguageTypeD(),
    listOf("036.방송콘텐츠 한국어-아시아어 번역 말뭉치"),
    showDatabase = false,
    showExamples = true,
    fast = true
)

In [22]:
class PreprocessorForParrellLanguageTypeE : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_TRANSLATION

    private val languageTable = mapOf(
        "en" to "영어",
        "ko" to "한국어",
        "ch" to "중국어",
        "jp" to "일본어",
    )

    private fun form(index: Int, sourceLanguage: String, targetLanguage: String, sourceText: String, targetText: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", translateInstructions[index % translateInstructions.size].replace("{a}", sourceLanguage).replace("{b}", "전문적인 ${targetLanguage}") + "\n\n" + sourceText))
        result.add(ChatRow("assistant", targetText))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .add("from") { "src_lang"<String>().let { languageTable[it]!! } }
            .add("to") { "tgt_lang"<String>().let { languageTable[it]!!} }
            .filter { standardVaildText("src_sentence"<String>()) && standardVaildText("tgt_sentence"<String>()) }
//            .explode("from")
//            .explode("to")
            .add("messages") {
                form(
                    index=index(),
                    sourceLanguage = "from"<String>(),
                    targetLanguage = "to"<String>(),
                    sourceText = ("src_sentence"<String>()).let { standardNormalizeText(it) },
                    targetText = ("tgt_sentence"<String>()).let { standardNormalizeText(it) }
                )
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["paragraph"].asFrameColumn().concat()
            dataFrame.let { it.remove("info") }.explode("sentences").flatten("sentences")
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForParrellLanguageTypeE(),
    listOf("053.한국어-다국어(영어 제외) 번역 말뭉치(기술과학)", "054.한국어-다국어 번역 말뭉치(기초과학)", "055.한국어-다국어 번역 말뭉치(인문학)"),
    showDatabase = false,
    showExamples = true,
    fast = true
)

In [23]:
class PreprocessorForParrellLanguageTypeF : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_TRANSLATION

    private val languageTable = mapOf(
        "en" to "영어",
        "ko" to "한국어",
        "ch" to "중국어",
        "jp" to "일본어",
    )

    private fun form(index: Int, sourceLanguage: String, targetLanguage: String, sourceText: String, targetText: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", translateInstructions[index % translateInstructions.size].replace("{a}", sourceLanguage).replace("{b}", "전문적인 ${targetLanguage}") + "\n\n" + sourceText))
        result.add(ChatRow("assistant", targetText))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .add("from") { "영어" }
            .add("to") { "한국어" }
            .add("from_text") {
                """
                | ## ${"invention_title_eng"<String>()}
                | ### Abstract
                | ${"astrt_cont_eng"<String>()}
                |
                | ### Description
                | ${"claim_eng"<String>()}
                """.trimMargin().trim()
            }
            .add("to_text") {
                """
                | ## ${"invention_title_kor"<String>()}
                | ### 초록
                | ${"astrt_cont_kor"<String>()}
                |
                | ### 설명
                | ${"claim_kor"<String>()}
                """.trimMargin().trim()
            }
            .filter { standardVaildText("from_text"<String>()) && standardVaildText("to_text"<String>()) }
//            .explode("from")
//            .explode("to")
            .let {
                // add reverse-pair
                it.mapToFrame {
                    "from" from it["to"]
                    "to" from it["from"]
                    "from_text" from it["to_text"]
                    "to_text" from it["from_text"]
                }
            }
            .add("messages") {
                form(
                    index=index(),
                    sourceLanguage = "from"<String>(),
                    targetLanguage = "to"<String>(),
                    sourceText = ("from_text"<String>()).let { standardNormalizeText(it) },
                    targetText = ("to_text"<String>()).let { standardNormalizeText(it) }
                )
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream).first()["labeled_data"] as DataFrame<*>
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForParrellLanguageTypeF(),
    listOf("155.산업정보 연계 주요국 특허 영-한 데이터"),
    showDatabase = false,
    showExamples = true,
    fast = true
)

In [24]:
class PreprocessorForBookCorpus : PreprocessorWithoutPreprocessing() {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, zipFile, stream ->
            if (zipFile.contains("INFO")) return@zipsJsonWalkAsync null

            val dataFrame = DataFrame.readJson(stream)["paragraphs"]
            dataFrame.asFrameColumn().concat().let {
                it
                    .filter {
                        // if any sentence is not valid, then it should be removed
                        ("sentences"<DataFrame<*>>()).map { "text"<String>().trim() }.all { standardVaildText(it, true) }
                    }
                    .add("text") { ("sentences"<DataFrame<*>>()).map { "text"<String>().trim() }.joinToString(" ") .let { standardNormalizeText(it) } }
                    .filterColumns("text")
            }
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForBookCorpus(),
    listOf("029.대규모 구매도서 기반 한국어 말뭉치 데이터"),
    showExamples = true,
    fast = true
)

In [25]:
class PreprocessorForNationalData : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { "corpus"<String>().replace(Regex("""<\|start\|>[^<]*<\|end\|>"""), "").let { standardNormalizeText(it) } }
            .filter { standardVaildText("text"<String>()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto(raw=true).filterNot { it.name.contains("TS_2") }.zipsJsonWalkAsync { _, _, stream ->
            val dataFrame = DataFrame.readJsonStr(stream.bufferedReader().readText().trim('﻿'))["data"]

            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForNationalData(),
    listOf("119.국가기록물 대상 초거대AI 학습을 위한 말뭉치 데이터"),
    showExamples = true
)

In [26]:
class PreprocessorForLLMData : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { runCatching { "contents"<String>().replace("<NAME>", "").replace("NAME", "").let { standardNormalizeText(it) } }.getOrDefault("") }
            .filter { standardVaildText("text"<String>()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto(raw=true).zipsJsonWalkAsync { _, zipFile, stream ->
            if (zipFile.contains("TS_02")) return@zipsJsonWalkAsync null // rlhf data

            val dataFrame = DataFrame.readJson(stream)["data_info"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForLLMData(),
    listOf("121.한국어 성능이 개선된 초거대AI 언어모델 개발 및 데이터"),
    showExamples = true,
    showDatabase = true
)

In [27]:
val summarizationInstructions = listOf(
    "주어진 글을 간결하게 요약해 주세요.",
    "이 글의 핵심을 간단히 정리해 주세요.",
    "이 글의 내용을 짧고 명료하게 요약하세요.",
    "글의 핵심을 요약하세요.",
    "글의 핵심 내용을 정리해 주세요.",
    "이 글을 간단하게 요약해 주세요.",
    "글의 주요 내용을 간단히 정리해 주세요.",
)

In [28]:
class PreprocessorForPaperSummarization : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_SUMMARIZATION
    
    private fun form(index: Int, sourceText: String, summary: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", summarizationInstructions[index % summarizationInstructions.size] + "\n\n" + sourceText))
        result.add(ChatRow("assistant", summary))

        return result
    }
    
    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("summary") { listOf(
            it["summary_entire"], it["summary_section"]
        ) }
            .explode("summary")
            .explode("summary")
            .add("messages") {
                form(
                    index=index(),
                    ("summary"<DataRow<*>>()["orginal_text"] as String).let { standardNormalizeText(it) },
                    ("summary"<DataRow<*>>()["summary_text"] as String).let { standardNormalizeText(it) }
                )
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["data"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForPaperSummarization(),
    listOf("018.논문자료 요약 데이터"),
    showExamples = true,
    fast = true
)

In [29]:
class PreprocessorForPaperSummarizationAsCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("summary") { listOf(
            it["summary_entire"], it["summary_section"]
        ) }
            .explode("summary")
            .explode("summary")
            .filter { standardVaildText("summary"<DataRow<*>>()["orginal_text"] as String) }
            .add("text") { ("summary"<DataRow<*>>()["orginal_text"] as String).let { standardNormalizeText(it) } }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["data"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForPaperSummarizationAsCorpus(),
    listOf("018.논문자료 요약 데이터"),
    showExamples = true,
    fast = true
)

In [30]:
class PreprocessorForBroadcastSummarization : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_SUMMARIZATION
    
    private fun form(index: Int, passage: String, summary: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", summarizationInstructions[index % summarizationInstructions.size] + "\n\n" + passage))
        result.add(ChatRow("assistant", summary))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter {
                ("Meta"<DataRow<*>>()["passage"] as String).let { standardVaildText(it) }
            }
            .add("messages") {
                form(
                    index=index(),
                    ("Meta"<DataRow<*>>()["passage"] as String),
                    ("Annotation"<DataRow<*>>()).let { 
                        "${it["Summary1"]} ${it["Summary2"]} ${it["Summary3"]}".trim().let { standardNormalizeText(it) }
                    }) 
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForBroadcastSummarization(),
    listOf("023.방송 콘텐츠 대본 요약 데이터"),
    showExamples = true,
    fast = true
)

In [31]:
class PreprocessorForBroadcastSummarizationAsCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS
    
    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter { ("Meta"<DataRow<*>>()["passage"] as String).let { standardVaildText(it) } }
            .add("text") { ("Meta"<DataRow<*>>()["passage"] as String).let { standardNormalizeText(it) } }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForBroadcastSummarizationAsCorpus(),
    listOf("023.방송 콘텐츠 대본 요약 데이터"),
    showExamples = true,
    fast = true
)

In [32]:
class PreprocessorForReportSummarization : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_SUMMARIZATION
    
    private fun form(index: Int, passage: String, summary: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", summarizationInstructions[index % summarizationInstructions.size] + "\n\n" + passage))
        result.add(ChatRow("assistant", summary))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter {
                ("Meta(Refine)"<DataRow<*>>()["passage"] as String).let { standardVaildText(it) }
            }
            .add("messages") {
                form(
                    index=index(),
                    ("Meta(Refine)"<DataRow<*>>()["passage"] as String).let {
                    standardNormalizeText(it.replace("\n   ", "<br>").replace("\n  ", " ").replace("<br>", "\n ")).trim()
                }, "Annotation"<DataRow<*>>().let {
                    "${it["summary1"]} ${it["summary2"]} ${it["summary3"]}".trim().let { standardNormalizeText(it) }
                }) 
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForReportSummarization(),
    listOf("022.요약문 및 레포트 생성 데이터"),
    showExamples = true,
    fast = true
)

In [33]:
class PreprocessorForReportSummarizationAsCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS
    
    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter { ("Meta(Refine)"<DataRow<*>>()["passage"] as String).let { standardVaildText(it) } }
            .add("text") { ("Meta(Refine)"<DataRow<*>>()["passage"] as String).let { standardNormalizeText(it.replace("\n   ", "<br>").replace("\n  ", " ").replace("<br>", "\n ")).trim() } }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForReportSummarizationAsCorpus(),
    listOf("022.요약문 및 레포트 생성 데이터"),
    showExamples = true,
    fast = true
)

In [34]:
class PreprocessorForTechSummarization : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_SUMMARIZATION
    
    private fun form(index: Int, context: String, summary: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", summarizationInstructions[index % summarizationInstructions.size] + "\n\n" + context))
        result.add(ChatRow("assistant", summary))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter { standardVaildText("context"<String>()) }
            .add("messages") { form(index(), "context"<String>().let { standardNormalizeText(it) }, "summary"<String>().let { standardNormalizeText(it) }) }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["dataset"]["context_info"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForTechSummarization(),
    listOf("153.기술과학 요약 데이터"),
    showExamples = true
)

In [35]:
class PreprocessorForTechSummarizationAsCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { "context"<String>().let { standardNormalizeText(it) } }
            .filter { standardVaildText("text"<String>()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["dataset"]["context_info"]
            dataFrame.asFrameColumn().concat()
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForTechSummarizationAsCorpus(),
    listOf("153.기술과학 요약 데이터"),
    showExamples = true
)

In [36]:
class PreprocessorForAbstractSummarization : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_SUMMARIZATION

    private fun form(index: Int, sourceText: String, summary: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", summarizationInstructions[index % summarizationInstructions.size] + "\n\n" + sourceText))
        result.add(ChatRow("assistant", summary))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter { standardVaildText("original_text"<String>()) }
            .add("messages") { form(index(), "original_text"<String>().let { standardNormalizeText(it) }, (("annotation"<DataRow<*>>()["original_summary"] as DataRow<*>)["human_abstractive_summary"] as DataRow<*>).let { standardNormalizeText(it["text"] as String) }) }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForAbstractSummarization(),
    listOf("157.추상 요약 사실성 검증 데이터"),
    showExamples = true
)

In [37]:
class PreprocessorForAbstractSummarizationAsCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { "original_text"<String>().let { standardNormalizeText(it) } }
            .filter { standardVaildText("text"<String>()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForAbstractSummarizationAsCorpus(),
    listOf("157.추상 요약 사실성 검증 데이터"),
    showExamples = true
)

In [38]:
class PreprocessorForBookSummarization : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_SUMMARIZATION

    private fun form(index: Int, passage: String, summary: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", summarizationInstructions[index % summarizationInstructions.size] + "\n\n" + passage))
        result.add(ChatRow("assistant", summary))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("messages") {
                form(
                    index=index(),
                    "passage"<String>().let { standardNormalizeText(it) },
                    "summary"<String>().let { standardNormalizeText(it) }
                )
            }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForBookSummarization(),
    listOf("도서자료 요약"),
    showExamples = true,
    fast = true
)

In [39]:
class PreprocessorForBookSummarizationAsCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") { "passage"<String>().let { standardNormalizeText(it) } }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForBookSummarizationAsCorpus(),
    listOf("도서자료 요약"),
    showExamples = true,
    fast = true
)

In [40]:
class PreprocessorForTextSummarization : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.TASK_SUMMARIZATION

    private fun form(index: Int, sourceText: String, summary: String): List<ChatRow> {
        val result = mutableListOf<ChatRow>()

        result.add(ChatRow("user", summarizationInstructions[index % summarizationInstructions.size] + "\n\n" + sourceText))
        result.add(ChatRow("assistant", summary))

        return result
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> = 
        data.add("messages") {
            form(
                index=index(),
                "text"<List<DataFrame<*>>>().joinToString("\n") {
                    it.map { it["sentence"] as String }.joinToString(" ")
                },
                "abstractive"<List<String>>().joinToString(" ").let { standardNormalizeText(it) }
            )
        }
            .filterColumns("messages")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream).first()["documents"] as DataFrame<*>
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForTextSummarization(),
    listOf("문서요약 텍스트"),
    showExamples = true,
    fast = true
)

In [41]:
class PreprocessorForTextSummarizationAsCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> = 
        data.add("_text") {
            "text"<List<DataFrame<*>>>().joinToString("\n") {
                it.map { it["sentence"] as String }.joinToString(" ")
            }
        }
            .filterColumns("_text")
            .rename("_text" to "text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream).first()["documents"] as DataFrame<*>
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForTextSummarizationAsCorpus(),
    listOf("문서요약 텍스트"),
    showExamples = true,
    fast = true
)

In [42]:
class PreprocessorForLaw : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS
    
    private data class LawInfo(
        val caseNm: String,
        val caseNo: String,
        val courtNm: String,
        val judgeDt: String
    )

    private fun infoFromDataRow(row: DataRow<*>): LawInfo = (row["info"] as DataRow<*>).let {
        LawInfo(
            caseNm = it["caseNm"] as String,
            caseNo = it["caseNo"] as String,
            courtNm = it["courtNm"] as String,
            judgeDt = it["judmnAdjuDe"] as String
        )
    }
    
    private data class LawBody(
        val disposal: String,
        val mentioned: String,
        val acusrAssrs: String,
        val dedatAssrs: String,
        val facts: String,
        val decision: String,
        val conclusion: String
    )

    private fun bodyFromDataRow(row: DataRow<*>): LawBody = LawBody(
        disposal = ((row["disposal"] as DataRow<*>)["disposalcontent"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") },
        mentioned = ((row["mentionedItems"] as DataRow<*>)["rqestObjet"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") },
        acusrAssrs = ((row["assrs"] as DataRow<*>)["acusrAssrs"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") }.trim(),
        dedatAssrs = ((row["assrs"] as DataRow<*>)["dedatAssrs"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") }.trim(),
        facts = ((row["facts"] as DataRow<*>)["bsisFacts"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") },
        decision = ((row["dcss"] as DataRow<*>)["courtDcss"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") },
        conclusion = ((row["close"] as DataRow<*>)["cnclsns"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") }
    )

    private fun form민사(row: DataRow<*>): String {
        val info = infoFromDataRow(row)
        val body = bodyFromDataRow(row)
        
        return """
        |# ${info.caseNm} [${info.courtNm} ${info.judgeDt} 선고 ${info.caseNo} 판결]
        |## 주문
        | ${body.disposal}
        |
        |## 청구취지 ${if (body.dedatAssrs.isNotBlank()) "및 항소취지" else ""}
        | ${body.mentioned}
        |
        |## 이유
        |### 기초사실
        | ${body.facts}
        |
        |### 원고의 주장
        | ${body.acusrAssrs}${if (body.dedatAssrs.isNotBlank()) "\n\n### 피고의 주장\n ${body.dedatAssrs}" else ""}
        |
        |## 판단
        | ${body.decision}
        |
        |## 결론
        | ${body.conclusion}
        """.trimMargin().trim().replace("  ", "")
    }
    
    private fun form형사(row: DataRow<*>): String {
        val info = infoFromDataRow(row)
        val body = bodyFromDataRow(row)
        
        val reason = when {
            body.acusrAssrs.isNotBlank() && body.dedatAssrs.isNotBlank() -> """
            |### 항소이유의 요지
            |#### 피고
            | ${body.dedatAssrs}
            |
            |#### 검사
            | ${body.acusrAssrs}
            """.trimMargin().trim().let { "\n${it}\n" }
            body.acusrAssrs.isNotBlank() -> """
            |### 검사의 항소이유의 요지
            | ${body.acusrAssrs}
            """.trimMargin().trim().let { "\n${it}\n" }
            body.dedatAssrs.isNotBlank() -> """
            |### 피고의 항소이유의 요지
            | ${body.dedatAssrs}
            """.trimMargin().trim().let { "\n${it}\n" }
            else -> ""
        }

        return """
        |# ${info.caseNm} [${info.courtNm} ${info.judgeDt} 선고 ${info.caseNo} 판결]
        |## 주문
        | ${body.disposal}
        |
        |## 청구취지 ${if (body.dedatAssrs.isNotBlank()) "및 항소취지" else ""}
        | ${body.mentioned}
        |
        |## 이유${reason}
        |### 기초사실
        | ${body.facts}
        |
        |## 판단
        | ${body.decision}
        |
        |## 결론
        | ${body.conclusion}
        """.trimMargin().trim().replace("  ", " ")
    }

    private fun form행정(row: DataRow<*>): String {
        val info = infoFromDataRow(row)
        val body = bodyFromDataRow(row)

        val reason = when {
            body.acusrAssrs.isNotBlank() && body.dedatAssrs.isNotBlank() -> """
            |### 항소이유의 요지
            |#### 피고
            | ${body.dedatAssrs}
            |
            |#### 원고
            | ${body.acusrAssrs}
            """.trimMargin().trim().let { "\n${it}\n" }
            body.acusrAssrs.isNotBlank() -> """
            |### 원고의 항소이유의 요지
            | ${body.acusrAssrs}
            """.trimMargin().trim().let { "\n${it}\n" }
            body.dedatAssrs.isNotBlank() -> """
            |### 피고의 항소이유의 요지
            | ${body.dedatAssrs}
            """.trimMargin().trim().let { "\n${it}\n" }
            else -> ""
        }

        return """
        |# ${info.caseNm} [${info.courtNm} ${info.judgeDt} 선고 ${info.caseNo} 판결]
        |## 주문
        | ${body.disposal}
        |
        |## 청구취지 ${if (body.dedatAssrs.isNotBlank()) "및 항소취지" else ""}
        | ${body.mentioned}
        |
        |## 이유${reason}
        |### 기초사실
        | ${body.facts}
        |
        |## 판단
        | ${body.decision}
        |
        |## 결론
        | ${body.conclusion}
        """.trimMargin().trim().replace("  ", " ")
    }
    
    private data class AgreementInfo(
        val clauseArticle: String,
        val comparisonProvision: String?,
        val illegalClause: String?,
        val relatedLaw: String?
    )
    
    private fun infoFromAgreement(row: DataRow<*>): AgreementInfo = AgreementInfo(
        clauseArticle = (row["clauseArticle"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") },
        comparisonProvision = if (row.containsKey("comProvision")) (row["comProvision"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") } else null,
        illegalClause = if (row.containsKey("illdcssBasiss")) (row["illdcssBasiss"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") } else null,
        relatedLaw = if (row.containsKey("relateLaword")) (row["relateLaword"] as List<*>).joinToString("\n ") { (it as String).trim().addSuffixIfNotExists(".") } else null
    )
    
    private fun form약관(row: DataRow<*>): String {
        val info = infoFromAgreement(row)
        
        if (info.comparisonProvision != null || info.illegalClause == null || info.relatedLaw == null) {
            // 유리
            
            return """
            |# 약관의 유불리 여부 판단
            |## 약관
            | ${info.clauseArticle}${if (info.comparisonProvision != null) "\n\n## 비교조항\n ${info.comparisonProvision}" else ""}
            |
            |## 판단
            | 위 약관은 고객에게 불리하지 않다.
            """.trimMargin().trim()
        } else {
            // 불리
            
            return """
            |# 약관의 유불리 여부 판단
            |## 약관
            | ${info.clauseArticle}
            |
            |## 관련법령
            | ${info.relatedLaw}
            |
            |## 판단
            | ${info.illegalClause}
            """.trimMargin().trim()
        }
    }

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.add("text") {
            when (this["type"] as String) {
                "판결문-민사" -> form민사(this)
                "판결문-형사" -> form형사(this)
                "판결문-행정" -> form행정(this)
                "약관" -> form약관(this)
                else -> "" // skip
            }
        }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val trainingLaw = file.auto().zipsJsonWalk { zip, zipFile, stream ->
            if (zip.contains("판결문")) {
                val dataFrame = DataFrame.readJson(stream)
                
                val lawType = when {
                    zipFile.contains("민사") -> "민사"
                    zipFile.contains("형사") -> "형사"
                    zipFile.contains("행정") -> "행정"
                    else -> return@zipsJsonWalk null
                }
                
                dataFrame.add("type") { "판결문-${lawType}" }
            } else {
                val dataFrame = DataFrame.readJson(stream)

                dataFrame.add("type") { "약관" }
            }
        }

        // merge all dataframes
        val data = trainingLaw.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForLaw(),
    listOf("019.법률, 규정 (판결서, 약관 등) 텍스트 분석 데이터"),
//    showDatabase = true,
    showExamples = true,
    fast = true
)

In [43]:
class PreprocessorForWebCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS
    
    private fun form(row: DataRow<*>): String = """
    |# ${(row["title"] as DataFrame<*>)["sentence"].toList().joinToString(" ") { it as String }}${(row["subtitle"] as DataFrame<*>)["sentence"].toList().joinToString(" ") { it as String }.let { 
        if (it.isNotBlank()) "\n## ${it}" else ""
    }}
    |
    |${(row["content"] as DataFrame<*>)["sentence"].toList().joinToString(" ") { it as String }}
    """.trimMargin().trim()
    
    private val replaceTarget = listOf(
        "[원문보기]", "(클릭)"
    )
    
    private val deleteTarget = listOf(
        "(이름)", "(전화번호)"
    )

    override fun preprocess(data: DataFrame<*>): DataFrame<*> = 
        data
            .filter { deleteTarget.none { target -> it["text"].toString().contains(target) } }
            .filter { standardVaildText(it["text"] as String) }

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream)["named_entity"]
            dataFrame.asFrameColumn().concat()
                .filter { 
                    // sentence-level filtering, if any sentence is not valid, then it should be removed
                    ("content"<DataFrame<*>>()).map { "sentence"<String>().trim() }.all { standardVaildText(it, true) }
                }
                .add("text") { form(this).let { standardNormalizeText(it.let { 
                    replaceTarget.fold(it) { acc, target -> acc.replace(target, "") }
                }) } }
                .filterColumns("text")
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForWebCorpus(),
    listOf("030.웹데이터 기반 한국어 말뭉치 데이터"),
    showExamples = true,
    fast = true
)

In [44]:
class PreprocessorForWebSpeechCorpus : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS_SPOKEN

    private fun form(row: DataRow<*>): String = (row["content"] as DataFrame<*>)["sentence"].toList().joinToString(" ") { it as String }.let { standardNormalizeText(it) }

    private val replaceTarget = listOf(
        "[원문보기]", "(클릭)"
    )

    private val deleteTarget = listOf(
        "(이름)", "(전화번호)", "(반사회적용어)"
    )

    override fun preprocess(data: DataFrame<*>): DataFrame<*> = data.filter { deleteTarget.none { target -> it["text"].toString().contains(target) } }

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream)["named_entity"]
            dataFrame.asFrameColumn().concat()
                .filter {
                    // sentence-level filtering, if any sentence is not valid, then it should be removed
                    ("content"<DataFrame<*>>()).map { "sentence"<String>().trim() }.all { standardVaildText(it, true) }
                }
                .add("text") { form(this).let { standardNormalizeText(it.let {
                    replaceTarget.fold(it) { acc, target -> acc.replace(target, "") }
                }) } }
                .filterColumns("text")
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForWebSpeechCorpus(),
    listOf("031.온라인 구어체 말뭉치 데이터"),
    showExamples = true,
    fast = true
)

In [45]:
val personaInstructions = listOf(
    "다음은 두 사람의 페르소나를 활용한 대화입니다. 두 사람의 페르소나에 대한 정보는 아래와 같습니다.\n\n{user}\n\n{character}\n\n이제 페르소나를 활용한 대화를 시작합니다.",
    "다음은 두 사람이 주어진 설정을 바탕으로 역할을 맡아 대화를 나누는 상황입니다. 두 사람의 역할에 대한 정보는 아래와 같습니다.\n\n{user}\n\n{character}\n\n이제 설정을 바탕으로 대화를 시작합니다.",
    "{user_name}과 {character_name}은 각자의 역할을 따라 대화를 나눕니다. {user_name}과 {character_name}에 대한 정보는 아래와 같습니다.\n\n{user}\n\n{character}\n\n이제 역할에 따른 대화를 시작합니다.",
    "역할극에 {user_name}과 {character_name}이 참여해 말하고 있습니다. {user_name}과 {character_name}에 대한 정보는 아래와 같습니다.\n\n{user}\n\n{character}\n\n이제 역할극을 시작합니다.",
    "다음은 {user_name}과 {character_name}의 대화입니다. {user_name}과 {character_name}에 대한 정보는 아래와 같습니다.\n\n{user}\n\n{character}\n\n이제 대화를 시작합니다."
)

class PreprocessorForPersonaChat : PreprocessorWithoutPreprocessing() {
    override val type: PreprocessorType = PreprocessorType.CHAT
    
    private fun formPersona(data: DataRow<*>, name: String): String {
        val persona = (data["persona"] as DataFrame<*>)
        
        val entries = persona.filter {
            if (it.containsKey("profile_major") && it.containsKey("profile_minor")) {
                val major = "profile_major"<String>()
                val minor = "profile_minor"<String?>()
                major.isNotBlank() && minor?.isNotBlank() ?: false
            } else {
                false
            }
        }.map { 
            val major = "profile_major"<String>().let { standardNormalizeText(it) }
            val minor = "profile_minor"<String>().let { standardNormalizeText(it) }
            " - ${major}: ${minor}"
        }.joinToString("\n ")
        
        return """
        |## ${name} 프로필
        |### 설명
        | ${persona.map { (it["profile"] as String).let { standardNormalizeText(it.addSuffixIfNotExists(".")) } }.joinToString(" ")}
        |
        |### 설정
        | ${entries}
        """.trimMargin().trim()
    }

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            val personas = ((dataFrame["info"] as DataRow<*>)["personas"] as DataFrame<*>)
            val user = formPersona(personas.first(), "유저")
            val character = formPersona(personas.last(),"캐릭터")

            val userId = (personas.first()["persona"] as DataFrame<*>)[0]["profile_id"] as Int
            
            val conversations = dataFrame["utterances"] as DataFrame<*>
            
            val result = mutableListOf<ChatRow>()

            result.add(ChatRow("system", personaInstructions[abs(conversations.hashCode()) % personaInstructions.size].replace("{user}", user).replace("{character}", character).replace("{user_name}", "유저").replace("{character_name}", "캐릭터")))
            
            conversations.forEach { 
                val isUser = "persona_id"<Int>() == userId
                
                val text = "text"<String>().let { standardNormalizeText(it) }
                
                result.add(ChatRow(if (isUser) "user" else "assistant", text))
            }

            result.mergeRepeating()
        }

        // merge all dataframes
        return dataFrameOf("messages")(*(training.toTypedArray()))
    }
}

declarePreprocessor(
    PreprocessorForPersonaChat(),
    listOf("044.페르소나 대화"),
    showExamples = true,
    fast = true
)

In [46]:
val topicInstructions = listOf(
    "다음은 {topic}에 대한 대화입니다. 당신은 {topic}에 대한 전문가이며 유저의 질문에 친절하고 정확하게 답변합니다. 이제 {topic}에 대한 대화를 시작합니다.",
    "당신은 {topic}에 대한 전문가입니다.",
    "{topic}에 대한 전문가로서 유저에게 {topic}에 대한 정보를 제공합니다. 이제 {topic}에 대한 대화를 시작합니다.",
    "{topic}에 대한 지식을 활용해 유저에게 도움이 되는 정보를 알려 줍니다. 이제 {topic}에 대한 대화를 시작합니다."
)

In [47]:
class PreprocessorForKnowledgeChat : PreprocessorWithoutPreprocessing() {
    override val type: PreprocessorType = PreprocessorType.CHAT

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            val topic = ((dataFrame["info"] as DataRow<*>)["topic"] as String).let { standardNormalizeText(it) }

            val conversations = dataFrame["utterances"] as DataFrame<*>

            val result = mutableListOf<ChatRow>()

            result.add(ChatRow("system", topicInstructions[abs(conversations.hashCode()) % topicInstructions.size].replace("{topic}", topic)))

            conversations.forEach {
                val isUser = "role"<String>() == "질문자"

                val text = "text"<String>().let { standardNormalizeText(it) }

                result.add(ChatRow(if (isUser) "user" else "assistant", text))
            }

            result.mergeRepeating()
        }

        // merge all dataframes
        return dataFrameOf("messages")(*(training.toTypedArray()))
    }
}

declarePreprocessor(
    PreprocessorForKnowledgeChat(),
    listOf("045.지식검색 대화"),
    showExamples = true,
    fast = true
)

In [48]:
val expertiseInstructions = listOf(
    "다음은 {topic}에 대한 대화입니다. 당신은 {topic}에 대한 전문가이며 유저의 질문에 친절하고 정확하게 답변합니다. 당신과 유저의 정보는 다음과 같습니다:\n\n{user}\n\n{character}\n\n이제 {topic}에 대한 대화를 시작합니다.",
    "당신은 {topic}에 대한 전문가입니다.",
    "{topic}에 대한 전문가로서 유저에게 {topic}에 대한 정보를 제공합니다. 당신과 유저의 정보를 참고해서 대답하세요.\n\n{user}\n\n{character}\n\n이제 {topic}에 대한 대화를 시작합니다.",
    "{topic}에 대한 지식을 활용해 유저에게 도움이 되는 정보를 알려 줍니다. 이제 {topic}에 대한 대화를 시작합니다."
)

In [49]:
class PreprocessorForExpertiseChat : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CHAT

    private val personaMap = mapOf(
        "gender" to "성별",
        "age" to "나이",
        "occupation" to "직업",
    )

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.explode("messages")

    private fun formPersona(data: DataRow<*>, key: String, name: String): String {
        val participant = (data["participantsInfo"] as DataRow<*>)[key] as DataRow<*>
        val info = (data["personaInfo"] as DataRow<*>).let {
            if (key == "speaker1") it["apprenticeInfo"] as DataRow<*> else it["wizardInfo"] as DataRow<*>
        }["personaFeatures"] as List<*>

        val entries = personaMap.map { (key, value) ->
            val entry = participant[key] as String
            if (entry.isNotBlank()) " - ${value}: ${entry}" else ""
        }.joinToString("\n ")

        return """
        |## ${name} 프로필
        |### 설명
        | ${info.joinToString(" ") { (it as String).addSuffixIfNotExists(".").replace("챗봇", "전문가") }}
        |
        |### 설정
        | ${entries}
        """.trimMargin().trim()
    }

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            val user = formPersona(dataFrame, "speaker1", "유저")
            val character = formPersona(dataFrame, "speaker2", "전문가")

            // check if there is another speaker in the data
            if ((dataFrame["participantsInfo"] as DataRow<*>).containsKey("speaker3")) return@zipsJsonWalkAsync null

            val conversations = dataFrame["sessionInfo"] as DataFrame<*>

            conversations.map {
                val dialogs = "dialog"<DataFrame<*>>()
                val topic = "sessionKeywords"<List<String>>().joinToString(", ")

                val result = mutableListOf<ChatRow>()

                result.add(
                    ChatRow(
                        "system",
                        expertiseInstructions[abs(dialogs.hashCode()) % expertiseInstructions.size].replace(
                            "{user}",
                            user
                        ).replace("{character}", character).replace("{topic}", topic)
                    )
                )

                dialogs.forEach {
                    val isUser = "speaker"<String>() == "speaker1"

                    val text = (it["utterance"] as String).let { standardNormalizeText(it) }

                    result.add(ChatRow(if (isUser) "user" else "assistant", text))
                }

                result.mergeRepeating()
            }
        }

        // merge all dataframes
        return dataFrameOf("messages")(*(training.toTypedArray()))
    }
}

declarePreprocessor(
    PreprocessorForExpertiseChat(),
    listOf("009.전문분야_기술과학_한국어 멀티세션 데이터", "010.전문분야_사회과학_한국어 멀티세션 데이터"),
    showExamples = true,
    fast = true
)

In [50]:
val empathyInstructions = listOf(
    "다음은 {relation} 관계에 있는 두 사람의 대화입니다. 당신은 {speaker_relation}의 말에 공감하며 대화합니다. 각자의 관계를 존중하며 대화를 이어나가세요. 상황: {situation} 이제 대화를 시작합니다.",
    "두 사람은 {relation} 관계에 있습니다. 당신은 {speaker_relation}의 말에 공감하며 대화합니다. 서로의 관계에 주목하고 상황에 적절한 대화를 이어나가세요.\n\n상황: {situation}\n\n이제 대화를 시작합니다.",
    "{relation} 관계에 있는 두 사람의 대화입니다. 당신은 {speaker_relation}의 말에 공감하며 대화합니다. 각자의 관계를 존중하며 말하세요.\n\n상황: {situation}",
    "당신은 {speaker_relation}의 말에 공감하며 대화합니다. 서로의 {relation} 관계를 존중하며 대화하세요.\n\n상황: {situation}\n\n",
    "당신은 주어진 상황에 맞춰 상대에게 공감하며 말합니다."
)

In [51]:
class PreprocessorForEmpathyChat : PreprocessorWithoutPreprocessing() {
    override val type: PreprocessorType = PreprocessorType.CHAT

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            val (relation, situation, speakerRelation) = (dataFrame["info"] as DataRow<*>).let {
                listOf(it["relation"] as String, it["situation"] as String, it["speaker_relation"] as String)
            }

            val conversations = dataFrame["utterances"] as DataFrame<*>

            val result = mutableListOf<ChatRow>()

            result.add(ChatRow("system", empathyInstructions[abs(conversations.hashCode()) % empathyInstructions.size].replace("{relation}", relation).replace("{situation}", situation).replace("{speaker_relation}", speakerRelation)))

            conversations.forEach {
                val isUser = "role"<String>() == "speaker"

                val text = "text"<String>().let { standardNormalizeText(it) }

                result.add(ChatRow(if (isUser) "user" else "assistant", text))
            }

            result.mergeRepeating()
        }

        // merge all dataframes
        return dataFrameOf("messages")(*(training.toTypedArray()))
    }
}

declarePreprocessor(
    PreprocessorForEmpathyChat(),
    listOf("046.공감형 대화"),
    showExamples = true,
    fast = true
)

In [52]:
class PreprocessorForMultisessionChat : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CHAT

    private val personaMap = mapOf(
        "gender" to "성별",
        "age" to "나이",
        "occupation" to "직업",
    )

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.explode("messages")

    private fun formPersona(data: DataRow<*>, key: String, name: String): String {
        val participant = (data["participantsInfo"] as DataRow<*>)[key] as DataRow<*>
        val info = (data["personaInfo"] as DataRow<*>).let { 
            if (key == "speaker1") it["clInfo"] as DataRow<*> else it["cpInfo"] as DataRow<*>
        }["personaFeatures"] as List<*>

        val entries = personaMap.map { (key, value) ->
            val entry = participant[key] as String
            if (entry.isNotBlank()) " - ${value}: ${entry}" else ""
        }.joinToString("\n ")
        
        return """
        |## ${name} 프로필
        |### 설명
        | ${info.joinToString(" ") { it as String }}
        |
        |### 설정
        | ${entries}
        """.trimMargin().trim()
    }

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalkAsync { _, z, stream ->
            val dataFrame = DataFrame.readJson(stream).first() as DataRow<*>
            val user = formPersona(dataFrame, "speaker1", "유저")
            val character = formPersona(dataFrame, "speaker2", "캐릭터")
            
            // check if there is another speaker in the data
            if ((dataFrame["participantsInfo"] as DataRow<*>).containsKey("speaker3")) return@zipsJsonWalkAsync null
            
            val conversations = dataFrame["sessionInfo"] as DataFrame<*>

            conversations.map {
                val dialogs = "dialog"<DataFrame<*>>()

                val result = mutableListOf<ChatRow>()

                result.add(ChatRow("system", personaInstructions[abs(dialogs.hashCode()) % personaInstructions.size].replace("{user}", user).replace("{character}", character)))

                dialogs.forEach {
                    val isUser = "speaker"<String>() == "speaker1"

                    val text = (it["utterance"] as String).let { standardNormalizeText(it) }

                    result.add(ChatRow(if (isUser) "user" else "assistant", text))
                }

                result.mergeRepeating()
            }
        }

        // merge all dataframes
        return dataFrameOf("messages")(*(training.toTypedArray()))
    }
}

declarePreprocessor(
    PreprocessorForMultisessionChat(),
    listOf("141.한국어 멀티세션 대화"),
    showExamples = true,
    fast = true
)

In [53]:
class PreprocessorForCommonSenseGeneration : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.CORPUS

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.explode("genSentences")
            .filter { "genSentences"<DataRow<*>>().let { it["label-scenes"] as String }.let { standardVaildText(it) } }
            .add("text") { "genSentences"<DataRow<*>>().let { it["label-scenes"] as String }.let { standardNormalizeText(it).trim() } }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)
            dataFrame
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForCommonSenseGeneration(),
    listOf("048.일반상식 문장 생성 데이터"),
    showExamples = true,
    fast = true
)

In [54]:
import java.nio.charset.Charset
import kotlin.io.path.bufferedReader

class PreprocessorForParrellDialect : Preprocessor {
    override val type: PreprocessorType = PreprocessorType.DIALECT

    private val locations = listOf(
        "강원도",
        "경상도",
        "충청도",
        "전라도",
        "제주도"
    )
    
    private fun String.punctuationCorrection() = 
        if (this.last() in correctPunctuation) this else this + "."

    private fun form(sourceLanguage: String, targetLanguage: String, sourceText: String, targetText: String): String = """
    |${sourceLanguage}: ${sourceText.punctuationCorrection()}
    |${targetLanguage}: ${targetText.punctuationCorrection()}
    """.trimMargin().trim()

    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data
            .filter { standardVaildText("from_text"<String>()) && standardVaildText("to_text"<String>()) }
            // swapping source and target language (explode) (source: dialect, target: standard(original) + source: standard, target: dialect)
            .let {
//                it.concat(
//                    dataFrameOf("from", "to", "from_text", "to_text")(
//                        it["to"].toList(),
//                        it["from"].toList(),
//                        it["to_text"].toList(),
//                        it["from_text"].toList()
//                    )
//                )
                it.concat(
                    it.mapToFrame {
                        "from" from it["to"]
                        "to" from it["from"]
                        "from_text" from it["to_text"]
                        "to_text" from it["from_text"]
                    }
                )
            }
            .add("text") { form("from"<String>(), "to"<String>(), "from_text"<String>(), "to_text"<String>()) }
//            .let { it.remove(*(it.columnNames() - listOf("from", "to", "from_text", "to_text")).toTypedArray()) }
            .filterColumns("text")

    override fun read(file: File): DataFrame<*> {
        val training = file.auto().zipsJsonWalk { zipFile, _, stream ->
            val dialect = locations.firstOrNull { zipFile.contains(it) } ?: return@zipsJsonWalk null
            val dataFrame = DataFrame.readJson(stream)

            (dataFrame
                .first()["transcription"] as DataRow<*>)
                .let {
                    // keep only dialect, standard field
                    it.df().remove(*(it.columnNames() - listOf("dialect", "standard")).toTypedArray())
                        .add("from") { "${dialect} 사투리" }
                        .add("to") { "표준어" }
                        .rename("dialect" to "from_text", "standard" to "to_text")
                }.first() 
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForParrellDialect(),
    listOf("139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)", "139-2.중·노년층 한국어 방언 데이터 (충청도, 전라도, 제주도)"),
    showDatabase = false,
    showExamples = true,
    fast = true
)

In [55]:
/**
 * [모두의 말뭉치](https://kli.korean.go.kr/corpus/)
 */
class PreprocessorForModuCorpusText : PreprocessorWithoutPreprocessing() {
    override val type: PreprocessorType = PreprocessorType.CORPUS
    
    private fun form(title: String, content: String): String = """
    |## ${title}
    |${content}
    """.trimMargin().trim()

    override fun read(file: File): DataFrame<*> {
        val training = listOf(file).zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["document"].asFrameColumn().concat()
            dataFrame
                .mapToFrame {
                    "text" from {
                        val df = "paragraph"<DataFrame<*>>().map { "form"<String>() }

                        form(
                            df[0],
                            df.drop(1).joinToString("\n").let { " " + standardNormalizeText(it) }
                        )
                    }
                }
                .filter { standardVaildText("text"<String>()) }
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForModuCorpusText(),
    listOf("NIKL_NEWSPAPER_2020_v1.1_JSON.zip", "NIKL_NEWSPAPER_2021_v1.0_JSON.zip", "NIKLNEWSPAPER_2022_v1.0_JSON.zip", "NIKL_NEWSPAPER_2023_JSON_v1.0.zip", "NIKL_NEWSPAPER_v2.0_JSON.zip", "NIKL_WRITTEN_v1.2_JSON.zip"),
    showExamples = true
)

In [56]:
import kotlin.collections.last as kLast

val personaTopicInstructions = listOf(
    "다음은 {relation} 관계에 있는 두 사람의 {topic}에 대한 대화입니다. 두 사람의 대화 내용에 주목하며 대화를 이어나가세요. 이제 대화를 시작합니다.",
    "두 사람은 {relation} 관계에 있습니다. {topic}에 대한 대화를 이어나가세요. 이제 대화를 시작합니다.",
    "{topic}에 대해 두 {relation} 관계에 있는 사람이 대화를 나눕니다. 각자의 관계를 존중하며 대화를 이어나가세요. 이제 대화를 시작합니다.",
    "{topic}에 대해 두 사람이 서로 말하고 있습니다. 주제에 어긋나지 않도록 자연스러운 대화를 이어나가세요. 이제 대화를 시작합니다.",
)

/**
 * [모두의 말뭉치](https://kli.korean.go.kr/corpus/)
 */
class PreprocessorForModuCorpusChat : PreprocessorWithoutPreprocessing() {
    override val type: PreprocessorType = PreprocessorType.CHAT

    private val personaMap = mapOf(
        "sex" to "성별",
        "age" to "나이",
        "occupation" to "직업",
    )
    
    override fun preprocess(data: DataFrame<*>): DataFrame<*> =
        data.explode("messages")

    private fun formPersona(data: DataRow<*>, key: Int, name: String): Pair<String, String> {
        val participant = ((data["metadata"] as DataRow<*>)["speaker"] as DataFrame<*>)[key] as DataRow<*>
        val id = participant["id"] as String

        val entries = personaMap.map { (key, value) ->
            val entry = participant[key] as String
            if (entry.isNotBlank()) " - ${value}: ${entry}" else ""
        }.joinToString("\n ")

        return """
        |## ${name} 프로필
        |### 설정
        | ${entries}
        """.trimMargin().trim() to id
    }

    override fun read(file: File): DataFrame<*> {
        val training = listOf(file).zipsJsonWalk { _, _, stream ->
            val dataFrame = DataFrame.readJson(stream)["document"].asFrameColumn().concat()

            dataFrame
                .filter {
                    // speaker must be two
                    ((it["metadata"] as DataRow<*>)["speaker"] as DataFrame<*>).rowsCount() == 2
                }
                .mapToFrame {
                    "messages" from {
                        val (user, userId) = formPersona(it, 0, "유저")
                        val (character, _) = formPersona(it, 1, "캐릭터")

                        val relation =
                            "metadata"<DataRow<*>>().run { "setting"<DataRow<*>>() }["relation"] as String
                        val topic = ("metadata"<DataRow<*>>()["topic"] as String).split(">").kLast().trim()

                        val result = mutableListOf<ChatRow>()

                        result.add(ChatRow("system", personaTopicInstructions[abs((relation + topic).hashCode()) % personaTopicInstructions.size].replace("{relation}", relation).replace("{topic}", topic)))

                        "utterance"<DataFrame<*>>().forEach {
                            val isUser = "speaker_id"<String>() == userId

                            val text = "form"<String>().let { standardNormalizeText(it) }

                            result.add(ChatRow(if (isUser) "user" else "assistant", text))
                        }

                        result.mergeRepeating()
                    }
                }
        }

        // merge all dataframes
        val data = training.concat()
        return data
    }
}

declarePreprocessor(
    PreprocessorForModuCorpusChat(),
    listOf("NIKL_DIALOGUE_2020_v1.4.zip", "NIKL_DIALOGUE_2021_v1.1.zip", "NIKL_DIALOGUE_2022_v1.0_JSON.zip"),
    showExamples = true
)

In [57]:

// find not preprocessed files

val notPreprocessed = files.filter { it !in preprocessors.keys }
notPreprocessed

[]

In [58]:
files

[009.전문분야_기술과학_한국어 멀티세션 데이터, 010.전문분야_사회과학_한국어 멀티세션 데이터, 016.행정 문서 대상 기계독해 데이터, 017.뉴스 기사 기계독해 데이터, 018.논문자료 요약 데이터, 019.법률, 규정 (판결서, 약관 등) 텍스트 분석 데이터, 021.도서자료 기계독해, 022.요약문 및 레포트 생성 데이터, 023.방송 콘텐츠 대본 요약 데이터, 025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터, 026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터, 027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터, 028.다국어 구어체 번역 병렬 말뭉치 데이터, 029.대규모 구매도서 기반 한국어 말뭉치 데이터, 030.웹데이터 기반 한국어 말뭉치 데이터, 031.온라인 구어체 말뭉치 데이터, 032.방송콘텐츠 한국어-영어 번역 말뭉치, 034.방송콘텐츠 한국어-유럽어 번역 말뭉치, 036.방송콘텐츠 한국어-아시아어 번역 말뭉치, 044.페르소나 대화, 045.지식검색 대화, 046.공감형 대화, 048.일반상식 문장 생성 데이터, 053.한국어-다국어(영어 제외) 번역 말뭉치(기술과학), 054.한국어-다국어 번역 말뭉치(기초과학), 055.한국어-다국어 번역 말뭉치(인문학), 119.국가기록물 대상 초거대AI 학습을 위한 말뭉치 데이터, 121.한국어 성능이 개선된 초거대AI 언어모델 개발 및 데이터, 141.한국어 멀티세션 대화, 150.숫자연산 기계독해 데이터, 152.기술과학 문서 기계독해 데이터, 153.기술과학 요약 데이터, 155.산업정보 연계 주요국 특허 영-한 데이터, 156.전문분야 영-한, 중-한 번역 말뭉치(식품), 157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터, 157.추상 요약 사실성 검증 데이터, NIKLNEWSPAPER_2022_v1.0_JSON.zip, NIKL_DIALOGUE_2020_v1.4.zip, NIKL_DIALOGUE_2021_v1.1.zip, NIKL_DIAL

In [67]:
fun grammarMistakeVisualize() {
    val df = dataFrameOf(
        "wrong" to commonGrammarMistakes.keys.toList(),
        "correct" to commonGrammarMistakes.values.toList(),
        "count" to commonGrammarMistakesStatistics.values.map { it.second }.toList()
    )

    println(df)
}

In [68]:
// all preprocessors are written.

val preprocessedDir = File("preprocessed").also { 
    if (!it.exists()) it.mkdirs()
}

files.forEach {
    val preprocessor = preprocessors[it] ?: return@forEach
    preprocessor.forEach { type, processor ->
        println("Preprocessing ${it} for ${type}")

        val file = File(preprocessedDir, "/${type}/${it}.jsonl").also {
            if (!it.parentFile.exists()) it.parentFile.mkdirs()
            if (!it.exists()) it.createNewFile()
            else return@forEach
        }
        
        processor.read(
            File("./datasets/${it}")
        ).let {
            if (processor is PreprocessorWithoutPreprocessing) it
            else processor.preprocess(it)
        }.let @OptIn(ExperimentalSerializationApi::class) @Suppress("UNCHECKED_CAST") {
            // check if text column exists
            if ("text" in it.columnNames()) {
                val textList = (it.toMap() as Map<String, List<String>>)["text"]!!
                println("Writing ${textList.size} rows to ${file}")
                // write by rows
                file.bufferedWriter().use { writer ->
                    textList.forEach { text ->
                        writer.write(Json.encodeToString(mapOf("text" to text)))
                        writer.newLine()
                    }
                }
            } else if ("messages" in it.columnNames()) {
//                Json.encodeToStream(it.toMap() as Map<String, List<List<ChatRow>>>, file.outputStream())
                val messages = (it.toMap() as Map<String, List<List<ChatRow>>>)["messages"]!!
                println("Writing ${messages.size} rows to ${file}")
                file.bufferedWriter().use { writer ->
                    messages.forEach { message ->
                        writer.write(Json.encodeToString(mapOf("messages" to message)))
                        writer.newLine()
                    }
                }
            } else {
                println("Text column does not exist in the dataframe.")
            }
        }
    }

    grammarMistakeVisualize()
}

println("All files are preprocessed.")

Preprocessing 009.전문분야_기술과학_한국어 멀티세션 데이터 for CHAT
Writing 120097 rows to preprocessed\CHAT\009.전문분야_기술과학_한국어 멀티세션 데이터.jsonl
    wrong correct count
  0    몇일      며칠  9232
  1  는 커녕     는커녕     0
  2  은 커녕     은커녕     0
  3    라고      라고    99
  4    부터      부터    66
  5   투성이     투성이     5
  6    짜리      짜리    27
  7    어치      어치     4
  8   할려고     하려고     6
  9   어떻해     어떡해     1
 10   어짜피     어차피     0
 11   곰곰히     곰곰이     6
 12    요세      요새    69
 13    금새      금세    61
 14     왠       웬   166
 15    웬지      왠지   164
 16  오랫만에    오랜만에    19
 17   일일히     일일이     6
 18  희안하다    희한하다     0
 19   희안한     희한한     0
...

Preprocessing 010.전문분야_사회과학_한국어 멀티세션 데이터 for CHAT
Writing 120761 rows to preprocessed\CHAT\010.전문분야_사회과학_한국어 멀티세션 데이터.jsonl
    wrong correct count
  0    몇일      며칠 15622
  1  는 커녕     는커녕     0
  2  은 커녕     은커녕     0
  3    라고      라고   248
  4    부터      부터   339
  5   투성이     투성이    14
  6    짜리      짜리    30
  7    어치      어치     9
  8   할려고     하려고    75
  9

In [69]:
dataFrameOf(
    "wrong" to commonGrammarMistakes.keys.toList(),
    "correct" to commonGrammarMistakes.values.toList(),
    "count" to commonGrammarMistakesStatistics.values.map { it.second }.toList()
)

wrong,correct,count
몇일,며칠,19130
는 커녕,는커녕,5924
은 커녕,은커녕,4872
라고,라고,106220
부터,부터,92914
투성이,투성이,6240
짜리,짜리,16132
어치,어치,11004
할려고,하려고,4898
어떻해,어떡해,447
